# 事前準備

- カタログ、スキーマの作成
  - 編集 > 置き換え > [ytsuchiya -> {ご自身の名前}]
- パフォーマンスチューニングのハンズオンで作成した、サーバレスSQLウェアハウス（Large）をコンピュートに選択

In [0]:
CREATE CATALOG IF NOT EXISTS ytsuchiya;
use catalog `ytsuchiya`;

In [0]:
CREATE SCHEMA IF NOT EXISTS ytsuchiya.perf_tuning;
use schema `perf_tuning`;

# リキッドクラスタリング適用前

In [0]:
-- Clustering Information が存在しないことを確認
DESCRIBE EXTENDED ytsuchiya.perf_tuning.trn_pos;

In [0]:
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT
  pos_salesdate,
  pos_storecd,
  pos_storename,
  pos_itemcd,
  pos_itemname,
  SUM(pos_salesamount) AS total_salesamount,
  SUM(pos_salesquantity) AS total_salesquantity
FROM ytsuchiya.perf_tuning.trn_pos
WHERE pos_salesdate >= DATE('2025-05-01')
  AND pos_salesdate <= DATE('2025-05-31')
  AND pos_kigyocd = '19'
GROUP BY
  pos_salesdate,
  pos_storecd,
  pos_storename,
  pos_itemcd,
  pos_itemname

# リキッドクラスタリング適用後

In [0]:
-- 最大4カラムまで指定可能
ALTER TABLE trn_pos CLUSTER BY (pos_salesdate, pos_kigyocd);

In [0]:
OPTIMIZE trn_pos FULL;

In [0]:
ANALYZE TABLE trn_pos COMPUTE STATISTICS FOR ALL COLUMNS;

In [0]:
-- 指定した列が Clustering Information に記載されていることを確認
DESCRIBE EXTENDED ytsuchiya.perf_tuning.trn_pos;

In [0]:
-- ワークショップではリザルトキャッシュの利用を必ずOFFにしておきます
set use_cached_result = false;

SELECT
  pos_salesdate,
  pos_storecd,
  pos_storename,
  pos_itemcd,
  pos_itemname,
  SUM(pos_salesamount) AS total_salesamount,
  SUM(pos_salesquantity) AS total_salesquantity
FROM ytsuchiya.perf_tuning.trn_pos
WHERE pos_salesdate >= DATE('2025-05-01')
  AND pos_salesdate <= DATE('2025-05-31')
  AND pos_kigyocd = '19'
GROUP BY
  pos_salesdate,
  pos_storecd,
  pos_storename,
  pos_itemcd,
  pos_itemname